## DIA 084: Mostrar Historial de Predicciones en una Tabla HTML con Flask + SQLite


Hoy aprenderás a mostrar el historial de predicciones que guardamos en la base de datos SQLite en el Día 83, usando una tabla HTML.
Mostraremos todos los registros en una página web ordenada y elegante con Bootstrap.

✅ ¿Qué aprenderás hoy?

📌 Consultar registros desde SQLite con SQLAlchemy.

📌 Mostrar datos dinámicamente en una tabla HTML.

📌 Crear una nueva ruta (/historial) para visualizar predicciones pasadas.

🗂️ Estructura del Proyecto
pgsql
Copiar
Editar
titanic_app/
├── app.py
├── model.pkl
├── db.sqlite3
└── templates/
    ├── index.html
    └── historial.html  ✅ NUEVA
🧩 1. Código Flask Actualizado (app.py)
Agregamos una nueva ruta /historial:

python
Copiar
Editar
from flask import Flask, render_template, request
from flask_sqlalchemy import SQLAlchemy
import joblib
import numpy as np
import datetime

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite3'
db = SQLAlchemy(app)

class Registro(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    f1 = db.Column(db.Float)
    f2 = db.Column(db.Float)
    f3 = db.Column(db.Float)
    f4 = db.Column(db.Float)
    prediccion = db.Column(db.String(50))
    fecha = db.Column(db.DateTime, default=datetime.datetime.utcnow)

modelo = joblib.load("model.pkl")

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predecir", methods=["POST"])
def predecir():
    try:
        f1 = float(request.form["f1"])
        f2 = float(request.form["f2"])
        f3 = float(request.form["f3"])
        f4 = float(request.form["f4"])

        datos = np.array([[f1, f2, f3, f4]])
        pred = modelo.predict(datos)[0]

        nuevo_registro = Registro(f1=f1, f2=f2, f3=f3, f4=f4, prediccion=str(pred))
        db.session.add(nuevo_registro)
        db.session.commit()

        return render_template("index.html", resultado=f"✅ Resultado: Clase {pred}")
    except:
        return render_template("index.html", resultado="❌ Error en los datos")

@app.route("/historial")
def historial():
    registros = Registro.query.order_by(Registro.fecha.desc()).all()
    return render_template("historial.html", registros=registros)

if __name__ == "__main__":
    db.create_all()
    app.run(debug=True)
🖼️ 2. Nueva Plantilla HTML (templates/historial.html)
html
Copiar
Editar
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Historial de Predicciones</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light">
<div class="container mt-5">
    <h2 class="text-center mb-4">📊 Historial de Predicciones</h2>
    <a href="/" class="btn btn-primary mb-3">← Volver</a>
    <div class="table-responsive">
        <table class="table table-bordered table-striped table-hover">
            <thead class="table-dark">
                <tr>
                    <th>#</th>
                    <th>Feature 1</th>
                    <th>Feature 2</th>
                    <th>Feature 3</th>
                    <th>Feature 4</th>
                    <th>Predicción</th>
                    <th>Fecha</th>
                </tr>
            </thead>
            <tbody>
                {% for r in registros %}
                <tr>
                    <td>{{ r.id }}</td>
                    <td>{{ r.f1 }}</td>
                    <td>{{ r.f2 }}</td>
                    <td>{{ r.f3 }}</td>
                    <td>{{ r.f4 }}</td>
                    <td>{{ r.prediccion }}</td>
                    <td>{{ r.fecha.strftime('%Y-%m-%d %H:%M:%S') }}</td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
    </div>
</div>
</body>
</html>
🔍 Líneas Clave Explicadas
Registro.query.order_by(...): Consulta todos los registros ordenados por fecha descendente.

{% for r in registros %}: Itera sobre los datos para construir la tabla.

r.fecha.strftime(...): Formatea la fecha para mostrarla con claridad.